In [ ]:
import cv2
import torch
import numpy as np

# Defining the initial secondary frame
prev_output = None  # To store the previous frame for optical flow comparison

# Downloading the MiDaS model
midas = torch.hub.load('intel-isl/MiDaS', 'MiDaS_small')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
midas.to(device)
midas.eval()

transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')
transform = transforms.small_transform

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Transforming the image
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    imgbatch = transform(img).to(device)

    with torch.no_grad():
        prediction = midas(imgbatch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode='bicubic',
            align_corners=False
        ).squeeze()

        output = prediction.cpu().numpy()

        # Normalize output
        output_normalized = cv2.normalize(output, None, 0, 255, cv2.NORM_MINMAX)
        output_normalized = np.uint8(output_normalized)

        # Apply Median filter to preserve edges and reduce noise
        output_filtered = cv2.medianBlur(output_normalized, 5)

    # If this is the first frame, skip comparison
    if prev_output is None:
        prev_output = output_filtered
        continue

    # Optical Flow for pixel-level movement detection (Farneback method)
    flow = cv2.calcOpticalFlowFarneback(prev_output, output_filtered, None, 
                                        0.5, 3, 15, 3, 5, 1.2, 0)
    mag, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    motion_map = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    motion_map = np.uint8(motion_map)

    # Segmenting into panes: left, middle, right
    height, width = motion_map.shape
    part_width = width // 3

    # Slice the motion map into three equal parts
    left = motion_map[:, :part_width]
    middle = motion_map[:, part_width:2 * part_width]
    right = motion_map[:, 2 * part_width:]

    # Calculating pixel-level motion in each pane
    left_motion = np.mean(left)
    middle_motion = np.mean(middle)
    right_motion = np.mean(right)

    # Dynamic threshold calculation based on standard deviation of motion
    motion_avg = np.mean([left_motion, middle_motion, right_motion])
    motion_std = np.std([left_motion, middle_motion, right_motion])
    dynamic_threshold = motion_avg + motion_std * 0.5  # Adjust factor as needed

    alert_messages = []

    # Classifying panes where significant motion is detected
    if left_motion > dynamic_threshold:
        alert_messages.append("Obstacle moving in the Left Pane")
        cv2.rectangle(frame, (0, 0), (part_width, height), (0, 0, 255), 2)
    if middle_motion > dynamic_threshold:
        alert_messages.append("Obstacle moving in the Middle Pane")
        cv2.rectangle(frame, (part_width, 0), (2 * part_width, height), (0, 0, 255), 2)
    if right_motion > dynamic_threshold:
        alert_messages.append("Obstacle moving in the Right Pane")
        cv2.rectangle(frame, (2 * part_width, 0), (width, height), (0, 0, 255), 2)

    # Display the alert messages for panes where obstacles are moving
    for message in alert_messages:
        print(message)

    # Update the previous frame for optical flow calculation in the next iteration
    prev_output = output_filtered

    # Display the output depth and motion maps alongside the frame with highlighted panes
    cv2.imshow('Depth Prediction (Filtered)', output_filtered)
    cv2.imshow('Motion Map', motion_map)
    cv2.imshow('CV2Frame', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()